In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import chardet

In [3]:
import fuzzywuzzy
from fuzzywuzzy import process

/Users/spencerfogelman/anaconda/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [5]:
data_clean = pd.read_csv('/Users/spencerfogelman/Downloads/WELLCOME/WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")

In [6]:
with open('/Users/spencerfogelman/Downloads/WELLCOME/WELLCOME_APCspend2013_forThinkful.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
print(result)

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}


In [7]:
data_clean.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [8]:
data_clean['Journal title'] = data_clean['Journal title'].str.lower()

In [9]:
data_clean['Journal title'].value_counts()[0:5]

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos genetics                       24
plos pathogens                      24
Name: Journal title, dtype: int64

In [10]:
journals = list(data_clean['Journal title'].value_counts()[0:5].index)
print(journals)

['plos one', 'journal of biological chemistry', 'neuroimage', 'plos genetics', 'plos pathogens']


In [11]:
data_clean['COST (£) charged to Wellcome (inc VAT when charged)'] = data_clean['COST (£) charged to Wellcome (inc VAT when charged)'].str.replace('£', '')

In [12]:
data_clean['COST (£) charged to Wellcome (inc VAT when charged)'] = data_clean['COST (£) charged to Wellcome (inc VAT when charged)'].str.replace('$', '')

In [13]:
data_clean['COST (£) charged to Wellcome (inc VAT when charged)'] = data_clean['COST (£) charged to Wellcome (inc VAT when charged)'].apply(float)

In [14]:
data_clean['Journal title'] = data_clean['Journal title'].str.strip()

In [15]:
titles = data_clean['Journal title'].unique()
# titles.sort()

In [16]:
fuzzywuzzy.process.extract("plos one", titles, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

[('plos one', 100),
 ('plos  one', 100),
 ('plos ntd', 75),
 ('plos 1', 71),
 ('plos genetics', 67),
 ('plos', 67),
 ('plos medicine', 67),
 ('plos pathogens', 64),
 ('plos biology', 60),
 ('plos medicine journal', 55)]

In [17]:
data_clean.loc[data_clean['Journal title'] == 'plos  one', 'Journal title'] = 'plos one'

In [18]:
data_clean.loc[data_clean['Journal title'] == 'plos 1', 'Journal title'] = 'plos one'

In [19]:
data_clean.loc[data_clean['Journal title'] == 'plos 1']

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)


In [20]:
fuzzywuzzy.process.extract("journal of biological chemistry ", titles, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

[('journal of biological chemistry', 100),
 ('journal biological chemistry', 95),
 ('the journal of biological chemistry', 94),
 ('jnl biological chemistry', 87),
 ('biological chemistry', 78),
 ('journal of biol chem', 78),
 ('biologicial chemistry', 77),
 ('journal of bioethical inquiry', 73),
 ('journal of cellular biochemistry', 73),
 ('european journal of cell biology', 70)]

In [21]:
data_clean.loc[data_clean['Journal title'] == 'journal biological chemistry', 'Journal title'] = 'journal of biological chemistry'

In [22]:
data_clean.loc[data_clean['Journal title'] == 'the journal of biological chemistry', 'Journal title'] = 'journal of biological chemistry'

In [23]:
data_clean.loc[data_clean['Journal title'] == 'jnl biological chemistry', 'Journal title'] = 'journal of biological chemistry'

In [24]:
data_clean.loc[data_clean['Journal title'] == 'journal of biol chem', 'Journal title'] = 'journal of biological chemistry'

In [25]:
data_clean.groupby(['Journal title'])['COST (£) charged to Wellcome (inc VAT when charged)'].mean()[journals]

Journal title
plos one                           37226.309848
journal of biological chemistry    32106.840154
neuroimage                          2215.168276
plos genetics                      84839.435000
plos pathogens                     84775.044583
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64

In [26]:
data_clean.groupby(['Journal title'])['COST (£) charged to Wellcome (inc VAT when charged)'].median()[journals]

Journal title
plos one                            901.81
journal of biological chemistry    1311.73
neuroimage                         2326.43
plos genetics                      1718.39
plos pathogens                     1600.52
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64

In [27]:
data_clean.groupby(['Journal title'])['COST (£) charged to Wellcome (inc VAT when charged)'].std()[journals]

Journal title
plos one                           185281.848567
journal of biological chemistry    173795.906174
neuroimage                            266.653947
plos genetics                      281865.707794
plos pathogens                     281885.544090
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64